In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses as lo_ss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import string
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as snsd
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.utils import class_weight
from sklearn.metrics import classification_report , roc_auc_score
import keras
import keras.backend as K
from google.colab import drive
from numpy import savetxt
from sklearn.metrics import confusion_matrix, accuracy_score

In [12]:
# Reading data from csv file

import os
github_raw_link="https://raw.githubusercontent.com/Aryansh085/LAP/main/858417d1-7d54-4115-a01b-fdda5e03ada3_testing_combined_rows4035_disc_1_0p9_MULTIPLY_preproces_155cols.csv"
df1=pd.read_csv((github_raw_link),header = None)
df1=np.array(df1)

df1.shape

# Making a new dataframe which contains only relevant features
df = df1[:,5:]
df = np.array(df)

# Standardizing data
from sklearn.preprocessing import StandardScaler
slr=StandardScaler()
slr.fit(df)
df = slr.transform(df)

# final dataframe
df=pd.DataFrame(df)
df.head()

# Making 3d array where each row contains last t rows information of the data
x = np.array(df)
# print(x.shape,y.shape)

# timestamp is defined here 
t = 5

# Storing output corresponding to the new dataframe and according to the timestamp
y=np.array(df1[:,0])
y=y[t:]

x_3d = []
for i in range(4035-t):
  tmp = []
  for j in range(t):
    tmp.append(x[j+i])
  x_3d.append(tmp)
x_3d = np.array(x_3d)
x_3d.shape

# making test and train input and output data for the lstm layer
total_size = x_3d.shape[0]

# assigning train-test ratio to split data in train-test
train_test_ratio=0.8
train_size = int(total_size * train_test_ratio)
test_size = total_size - train_size

In [13]:
# these array will contain test and train data
x_train, y_train, x_test, y_test = [], [], [], []

# Random assignment of data : filling the test and train data randomly
for i in range(x_3d.shape[0]):
  feature = []
  corr_output = []

  no = np.random.randint(100)

  if(no < 80):
    feature = x_3d[i]
    corr_output = y[i]

    x_train.append(feature) 
    y_train.append(corr_output) 

  else:
    feature = x_3d[i]
    corr_output = y[i]

    x_test.append(feature) 
    y_test.append(corr_output) 

# changing each to np array
main_x_train = np.array(x_train)
main_y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(main_x_train.shape, main_y_train.shape)
print(x_test.shape, y_test.shape)

(3203, 5, 150) (3203,)
(827, 5, 150) (827,)


## Saved model importing and predicting through it here

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import keras 

#import keras backend
import keras.backend as K

# f1-score function
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision + recall + K.epsilon())
    return f1_val

In [16]:
from tensorflow.keras.models import load_model
model=load_model("/content/drive/MyDrive/Colab Notebooks/LAP_project/model.h5")
# model=load_model("/content/drive/MyDrive/Colab Notebooks/model.h5")

# output of the model
test_output_1 = model.predict(x_test, verbose=1)
test_output_1 = np.array([np.argmax(i)  for i in test_output_1])
test_output_1.shape

# printing confusion matrix and accuracy score
print("confusion matrix : \n", confusion_matrix(y_test,test_output_1))
test_output = pd.DataFrame(test_output_1)
print("accuracy score : ", accuracy_score(y_test,test_output_1))

# printing f1-score
print(get_f1(y_test.astype('float32'), test_output_1.astype('float32')))

26/26 [==============================] - 5s 135ms/step
confusion matrix : 
 [[363  13  12]
 [ 11 194   1]
 [  7   5 221]]
accuracy score :  0.9407496977025392
tf.Tensor(0.95141244, shape=(), dtype=float32)


In [17]:
k = model.predict(x_test[50:80])
k

1/1 [==============================] - 0s 75ms/step


array([[5.58994412e-01, 3.84313345e-01, 2.27347583e-01],
       [9.48219359e-01, 8.19555763e-03, 1.00790372e-03],
       [2.45850682e-02, 9.82402444e-01, 7.44045451e-02],
       [6.25645936e-01, 1.67085856e-01, 9.99993980e-01],
       [6.35502636e-01, 1.39125362e-01, 9.99998927e-01],
       [5.51707804e-01, 2.40523577e-01, 9.99953210e-01],
       [4.56615329e-01, 4.15303200e-01, 9.94307578e-01],
       [5.73506594e-01, 2.22324729e-01, 9.99963343e-01],
       [9.15965438e-01, 1.91051513e-02, 3.47520388e-03],
       [8.77600074e-01, 3.74604948e-02, 9.40246787e-03],
       [5.78029513e-01, 3.30420583e-01, 9.44004506e-02],
       [9.88339663e-01, 6.56305288e-04, 2.58855453e-05],
       [9.32153285e-01, 1.31112291e-02, 2.00117403e-03],
       [3.31098586e-02, 9.75475669e-01, 8.74338299e-02],
       [2.97072437e-02, 9.78314519e-01, 8.34410712e-02],
       [7.66608417e-02, 9.28381503e-01, 3.96512866e-01],
       [5.00749797e-02, 9.63229537e-01, 1.58110023e-01],
       [2.37020124e-02, 9.83214

In [9]:
np.argmax(k,axis=1)

array([1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1,
       0, 2, 0, 1, 2, 2, 2, 0])

In [10]:
y_test[50:80]

array([1., 1., 2., 2., 0., 0., 0., 0., 0., 1., 1., 2., 2., 2., 2., 0., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 0., 2., 2., 2., 0.])